In [1]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import date
import statistics

In [2]:
def check_channel(link):
    # extracts number of videos, average view count, and median view count for past 10 videos
    try:
        html = urllib.urlopen(link + "/videos")
        soup2 = BeautifulSoup(html.read())
    except:
        return ("ERR0", "ERR0", "ERR0")
    
    count = 0
    views_list = []
    
    #print(soup2)
    try:
        scripts = soup2.find_all('script')
        #for script in scripts:
            # print("\n\n", script)
        vids = json.loads(scripts[27].get_text()[31:-110])
        # print(vids)
        vid_list_1 = vids["contents"]["twoColumnBrowseResultsRenderer"]["tabs"][1]["tabRenderer"]
        vid_list_2 = vid_list_1["content"]["sectionListRenderer"]["contents"][0]["itemSectionRenderer"]["contents"]
        video_list = vid_list_2[0]["gridRenderer"]["items"]
    except:
        return ("ERR1", "ERR1", "ERR1")
    
    try:
        for video in video_list:
            vid_info = video["gridVideoRenderer"]
            upload_time = vid_info["publishedTimeText"]["simpleText"]

            # check upload date - stop loop once 10 videos have been reached
            if count == 10: break

            # count number of videos in last month
            count += 1


            # get view count for each video
            # extract all characters except for " views" and convert to integer
            views = int(vid_info["viewCountText"]["simpleText"][:-6].replace(",", ""))

            views_list.append(views)
    except:
        return ("ERR2", "ERR2", "ERR2")
    
    # error handling if no videos are found in past month
    try:
        mean_views = statistics.mean(views_list)
        median_views = statistics.median(views_list)
    except:
        return ("ERR3", "ERR3", "ERR3")
    
    # print(link, "\t", count, mean_views, median_views)
    
    return (count, mean_views, median_views)

def round_5000(views):
    try:
        ans = int(round(float(views) / 5000) * 5)
    except:
        ans = "ERR"
    return ans
def calculate_money(views):
    try:
        ans = int(float(views) * 5 * 6.2)
    except:
        ans = "ERR"
    return ans

In [3]:
print(check_channel("https://www.youtube.com/c/BleepinjeepPage"))
#print(check_channel("http://www.youtube.com/channel/UCPGhzlxltApd7GEngwgdPig"))


(10, 79461.4, 23439.5)


In [5]:
contacts = pd.read_csv("/Users/basse/OneDrive/Documents/Modern Media Merch/contact names.csv")
print(contacts)

                       Name
0              John Skinner
1      Japan Food Adventure
2               Alliesrasza
3                NoobHunter
4                  amandabb
...                     ...
13842            gamerZsoul
13843           FÁBIO GAMER
13844            tombino546
13845         NtenseFit Way
13846      BOSSO PRODUCTION

[13847 rows x 1 columns]


In [6]:
"John Skinner" in contacts.values
# date.today().strftime("%Y-%m-%d")

True

In [ ]:
# TODO adapt code to use Scrapy - potentially much faster and feasible to run on cloud
# Run current code on pythonanywhere and see how much cpu time it takes for 10 pages

In [99]:
# https://www.channelcrawler.com/eng

# initialize dataframe
df = pd.DataFrame(columns = ["Name", "Subscribers", "Genre", "Link", "Num videos", 
                            "Median views", "Projected Earnings", "Mean views", "Date Added"])

# id for search
number = 4062 # all genres 1k + 
# number = 1553 # all genres 10k+
# number = 255613 all genres 25k - 3 mil
# number = 197785 all genres 100k - 3 mil
# number = 195112 gaming 100k - 3 mil
# number = 132065 gaming 100k - 2mil
# number = 111710  gaming old
# number = 136782 all genres
search_link = "https://www.channelcrawler.com/eng/results/" + str(number)

# get current list of contacts
contacts = pd.read_csv("/Users/basse/OneDrive/Documents/Modern Media Merch/contact names.csv")
for page in range(1, 51):
    # loop through each page of search results
    print(page)
    html = urllib.urlopen(search_link + "/page:" + str(page))
    soup = BeautifulSoup(html.read())

    # print(soup.body)

    # extract channel items from webpage
    channels = soup.find_all("div", class_="channel col-xs-12 col-sm-4 col-lg-3")
    # print(channels)

    # for each channel
    # get channel name, link, genre, and subscribers from channel crawler
    # visit youtube page and extract number of videos, average views per video, and max views in the last month
    for channel in channels:
        try:
            name = channel.find("a").get("title")

            # check if channel name is already in contact list, skip if so
            if name in contacts.values: continue

            print(name)

            link = channel.find("a").get("href")
            genre = channel.find_all("small")[0].get_text().strip()
            subscribers = channel.find_all("small")[1].get_text().strip()[:4]


            print("\t", link)


            num_vids, mean_views, median_views = check_channel(link)
            
            # round to nearest 5000
            mean_views = round_5000(mean_views)
            median_views = round_5000(median_views)
            projected_earns = calculate_money(median_views)

            print("\t", median_views, "k")

            df = df.append({"Name": name, "Link": link + "/about", "Genre": genre,
                            "Subscribers": subscribers, "Num videos": num_vids,
                           "Mean views": mean_views, "Median views": median_views, 
                            "Projected Earnings" : projected_earns,
                           "Date Added": date.today().strftime("%Y-%m-%d")},
                           ignore_index = True)

        except:
            # if fails for any reason, move on to next channel
            continue

        # print(df)
df.to_csv("/Users/basse/OneDrive/Documents/Modern Media Merch/new contact data v2.csv", mode = "a", 
      index = False, header = False)

contact_list = contacts.merge(df.loc[:, "Name"], "outer")
contact_list.to_csv("/Users/basse/OneDrive/Documents/Modern Media Merch/contact names.csv", index = False)

    
print(len(df.index))   
print(df)

1
Ø²Ù…Ø§Ù„Ùƒ Ø§Ù„ÙŠÙˆÙ…
2
3
Virna Huerta
4
5
kim soc Camera
6
7
Alpha Venom
8
9
Pablo and Co
10
11
12
13
14
15
Rina in London
16
Sen
17
SOKHA RFA
18
19
20
21
22
23
24
25
26
27
28
Sexy dance official
29
30
31
32
33
34
35
36
37
djofer officiel
38
SKREEK
39
Khadija Reeves
40
41
Kresnt
42
43
SmileyMe
44
45
46
47
48
49
50
SidLyrics SL
0
Empty DataFrame
Columns: [Name, Subscribers, Genre, Link, Num videos, Median views, Projected Earnings, Mean views, Date Added]
Index: []


In [ ]:
df

In [ ]:
df["Revenue"] = df["Average views last month"] * 0.005 * 4.77

In [ ]:
# contact_list = pd.concat(df["Name"], contacts.values)
# df.loc[:, "Name"]
df.to_csv("/Users/Able Care/Documents/Modern Media Merch/new contact data.csv", mode = "a", header = False)

contact_list = contacts.merge(df.loc[:, "Name"], "outer")
contact_list.to_csv("/Users/Able Care/Documents/Modern Media Merch/contact names.csv")

## 7. More on Web Scraping
* **BeautifulSoup** Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* **Selenium:** https://selenium-python.readthedocs.io/index.html
    * Web automation and scraping; dynamic GET and POST requests; can interact with dynamic web pages, forms, etc.
* **Scrapy:** https://scrapy.org/
    * Optimized web crawling tasks

In [5]:
# one-off update code
contact_list = pd.read_clipboard()

contact_list["Channel Link"] = contact_list["Channel Link"].str.replace("/about", "")
contact_list_exp = contact_list

def get_views(link):
    return check_channel(link)

contact_list_exp["Vid Count"] = pd.Series()
contact_list_exp["Mean Views"] = pd.Series()
contact_list_exp["Median Views"] = pd.Series()

contact_list_exp["Vid Count"], contact_list_exp["Mean Views"], contact_list_exp["Median Views"] = zip(*contact_list_exp["Channel Link"].map(get_views))

# round views, calculate projected earnings, add about to end of link
contact_list_exp["Mean Views"] = contact_list_exp["Mean Views"].apply(round_5000)
contact_list_exp["Median Views"] = contact_list_exp["Median Views"].apply(round_5000)
contact_list_exp["Projected Money"] = contact_list_exp["Median Views"].apply(calculate_money)
contact_list_exp["Channel Link"] = contact_list_exp["Channel Link"] + "/about"
contact_list_exp.to_clipboard()

C:\Users\basse\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\basse\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\basse\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':


In [ ]:
# migration to scrapy

In [1]:
import re
import statistics

from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

import pandas as pd

In [2]:
class ChannelCrawlerSpider(scrapy.Spider):
    name = "channel"
    
    number = 4062
    base_url = "https://www.channelcrawler.com/eng/results/" + str(number) + "/page:"
    
    channel_crawler_links = []
    for i in range(1, 3):
        channel_crawler_links.append(base_url + str(i))
    
    start_urls = channel_crawler_links
    custom_settings = { 'DOWNLOAD_DELAY': 1 }
    
    def parse(self, response):
        # extract channel links from page
        links = response.xpath('//*[@id="main-content"]//div//div//h4//a/@href').getall()     
        links = [link + "/videos" for link in links]
        '''
        channel_names = response.xpath('//*[@id="main-content"]//div//div//h4//a/text()').getall()    
        genres = response.xpath('//*[@id="main-content"]//div//div//small//b/text()').getall()
        
        # get rid of example videos which can also be picked up
        text = "Example"
        genres_cleaned = [x.strip() for x in genres if text not in x]
        
        subcount = response.xpath('//*[@id="main-content"]//div//div//p//small/text()').getall()
        # restrict strings to those containing subscriber, then extract text before the space (subscriber count)
        text = "Subscribers"
        subcount_cleaned = [x.strip().split(" ")[0] for x in subcount if text in x]
        '''
        yield from response.follow_all(links, self.parse_channel)
        
    def parse_channel(self, response):
        # extract view counts from youtube videos 
        view_data = response.xpath("//script[contains(., 'viewCount')]/text()").extract()[0]
        view_list = re.findall('shortViewCountText":{"simpleText":"(.+?) view', view_data)
        view_list_cleaned = []
        
        # drop unnecessary items from view list
        for view in view_list:
            if len(view) < 7: 
                # account for k vs m
                if view[-1] == "K":
                    view_list_cleaned.append(float(view[0:-1]) * 1000)
                elif view[-1] == "M":
                    view_list_cleaned.append(float(view[0:-1]) * 1000000)
                else:
                    view_list_cleaned.append(float(view))
            
            if len(view_list_cleaned) == 10: break
        
        median_views = statistics.median(view_list_cleaned)
        num_videos = len(view_list_cleaned)
        
        # get channel name, subscriber count
        channel_data = str(response.body)
        channel_name = re.findall('<meta name="title" content="(.+?)">', channel_data)[0]
        sub_count = re.findall('"subscriberCountText":{"simpleText":"(.+?) subscribers', channel_data)[0]
        
        print(channel_name, sub_count, response.url, median_views, num_videos, sep = "\t")
        
        yield{
            "Channel Name" : channel_name,
            "Channel link" : response.url,
            "Subscribers": sub_count,
            "Median Views": median_views,
            "Number of videos": num_videos
        }
            
        

In [3]:
if __name__ == "__main__":
    runner = CrawlerRunner({
        'FEED_URI': 'youtuberdata.csv',
        'FEED_FORMAT': 'csv',
    })
    
    d = runner.crawl(ChannelCrawlerSpider)
    d.addBoth(lambda _: reactor.stop())
    reactor.run()

C:\Users\basse\Anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:239: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)


Chris Lambert	5.04K	https://www.youtube.com/channel/UCjWIxEQuYgWKq0GHMW2IbOw/videos	62.0	10
Samsul Bhe	15.7K	https://www.youtube.com/channel/UCIEouqUGMEANz6WBRiP3Kfg/videos	57.0	10
Josh T	2.22K	https://www.youtube.com/channel/UChQeacYZdpxjQbh9EsO9r-w/videos	521.5	10
powerfulukltd	73.3K	https://www.youtube.com/channel/UCjCbH52t-y0TPd6Y4CElvFw/videos	3950.0	10
RBLstylelife	28.2K	https://www.youtube.com/channel/UCBFgGNi7zmRUPy9Iv8S6Flw/videos	708.0	10
PALE1080	33.9K	https://www.youtube.com/channel/UCzDXQ7i0mR5_PvT56YpPOXA/videos	6150.0	10
ANGEL PEQUE\xc3\x91O	15.5K	https://www.youtube.com/channel/UCNS6_G-SajjJYq7Nb98m82w/videos	349.0	10
Tonny L Nielsen	3.11K	https://www.youtube.com/channel/UCDzfexBSQCc0zssZPxgN83A/videos	103.5	10
PD	6.9K	https://www.youtube.com/channel/UC4O9wfeYvwwneoelzEiwVqA/videos	598.0	10
CBS Denver	47.5K	https://www.youtube.com/channel/UC_-gT7OYiRCK9Sp8SIv9WgQ/videos	68.0	10
MrcreeperGam1ng	2.77K	https://www.youtube.com/channel/UCVqL7pZqLUwU6l6SJUVqImQ/videos	298.5	1

In [3]:
# initialize dataframe
df = pd.DataFrame(columns = ["Name", "Subscribers", "Genre", "Link", "Num videos", 
                            "Median views", "Projected Earnings", "Mean views", "Date Added"])